In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [4]:
def get_model_data(context_audio_features, audio_features):
    model_data = pd.DataFrame(columns=['context_audio_feature', 'audio_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
        audio_key = row["SCENE"] + "_u.wav"
        context_audio_key = row["SCENE"] + "_c.wav"
        model_data = model_data.append({'context_audio_feature': context_audio_features[context_audio_key],
                                    'audio_feature': audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [5]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data

In [15]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_audio_feature']
            ctxt_features = self.data.loc[index, 'padded_context_audio_feature']
            final_features = np.vstack((ctxt_features, features))
            a=np.empty((36,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((final_features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_audio_feature']
            ctxt_features = self.data.loc[index, 'padded_context_audio_feature']
            final_features = np.vstack((ctxt_features, features))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 18
output_size = 2
input_size_sid = 690
input_size_sd = 691
n_layers = 1

In [7]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')
with open('../../audio_features/feat_dict_context_librosa_lld.pickle', 'rb') as f:
    librosa_context_audio_features = pickle.load(f, encoding='latin1')

    
model_data = get_model_data(librosa_context_audio_features, librosa_audio_features)  
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,context_audio_feature,audio_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[-637.1869506835938, -527.0845538153164, -543...","[[-386.6164855957031, -649.6673512776692, -633...",0.0,NONE,SHELDON,25
1,"[[-625.8624267578125, -403.14568843397984, -38...","[[-255.5221405029297, -484.69307309105284, -52...",0.0,NONE,PENNY,15
2,"[[-500.3988952636719, -396.36312383413315, -22...","[[-569.0548095703125, -381.4147456242488, -221...",0.0,NONE,RAJ,21
3,"[[-313.1777038574219, -228.85994769251624, -35...","[[-237.61074829101562, -211.002773845897, -382...",1.0,PRO,HOWARD,7
4,"[[-337.88116455078125, -420.55431980762665, -3...","[[-530.5701293945312, -374.83951552370763, -42...",0.0,NONE,SHELDON,25


In [8]:
desired_length = 18
desired_length_ctxt = 18
train_data, test_data = get_train_test_split(model_data, ['context_audio_feature', 'audio_feature', 'speaker_encode'], 'sarcasm', 'sarcasm')
gru_train = train_data.copy()
gru_test = test_data.copy()
gru_train.reset_index(drop=True, inplace = True)
gru_test.reset_index(drop=True, inplace = True)

        
gru_train['padded_audio_feature'] = gru_train.loc[:, 'audio_feature']
for index, row in gru_train.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    gru_train.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

gru_train['padded_context_audio_feature'] = gru_train.loc[:, 'context_audio_feature']
for index, row in gru_train.iterrows():
    data_array = row['padded_context_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length_ctxt:
            arr = np.pad(arr, (0, desired_length_ctxt - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    gru_train.at[index, "padded_context_audio_feature"] = np.transpose(np.array(new_array))

gru_test['padded_audio_feature'] = gru_test.loc[:, 'audio_feature']
for index, row in gru_test.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    gru_test.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

gru_test['padded_context_audio_feature'] = gru_test.loc[:, 'audio_feature']
for index, row in gru_test.iterrows():
    data_array = row['padded_context_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length_ctxt:
            arr = np.pad(arr, (0, desired_length_ctxt - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    gru_test.at[index, "padded_context_audio_feature"] = np.transpose(np.array(new_array))

gru_train["sarcasm"] = gru_train["sarcasm"].astype('int').to_numpy()
gru_test["sarcasm"] = gru_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 3) (961,) Test:  ((241, 3), (241,))


### Speaker Independent

In [9]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_context_audio_feature', 'padded_audio_feature', 'sarcasm']], False)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_context_audio_feature', 'padded_audio_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())

In [10]:
gru = GRUNetSID(input_size_sid, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSID(
  (gru): GRU(690, 18, batch_first=True)
  (fc): Linear(in_features=18, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [11]:
n_epochs = 501
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "fnnmodel.pt")
#         test_min_loss = test_loss
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits = 4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5021    1.0000    0.6685       121
           1     0.0000    0.0000    0.0000       120

    accuracy                         0.5021       241
   macro avg     0.2510    0.5000    0.3343       241
weighted avg     0.2521    0.5021    0.3356       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5400    0.6694    0.5978       121
           1     0.5604    0.4250    0.4834       120

    accuracy                         0.5477       241
   macro avg     0.5502    0.5472    0.5406       241
weighted avg     0.5502    0.5477    0.5408       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.5972    0.7107    0.6491       121
           1     0.6392    0.5167    0.5714       120

    accuracy                         0.6141       241
   macro avg     0.6182    0.6137    0.6102       241
weighted avg     0.6181    0.6141    0.6104  

Epoch: 500
              precision    recall  f1-score   support

           0     0.5652    0.5372    0.5508       121
           1     0.5556    0.5833    0.5691       120

    accuracy                         0.5602       241
   macro avg     0.5604    0.5603    0.5600       241
weighted avg     0.5604    0.5602    0.5599       241



### Speaker Dependent

In [16]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_context_audio_feature', 'padded_audio_feature', 'speaker_encode','sarcasm']], True)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_context_audio_feature', 'padded_audio_feature', 'speaker_encode','sarcasm']], True)

num_of_workers = 0
batch_size = 31
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last=True
)

test_loader_epoch = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())

In [17]:
gru = GRUNetSD(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSD(
  (gru): GRU(691, 18, batch_first=True)
  (fc): Linear(in_features=18, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [18]:
n_epochs = 501
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "fnnmodel.pt")
#         test_min_loss = test_loss
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(gru, test_loader_epoch, gru_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits = 4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.6031    0.6529    0.6270       121
           1     0.6182    0.5667    0.5913       120

    accuracy                         0.6100       241
   macro avg     0.6106    0.6098    0.6091       241
weighted avg     0.6106    0.6100    0.6092       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5714    0.7273    0.6400       121
           1     0.6207    0.4500    0.5217       120

    accuracy                         0.5892       241
   macro avg     0.5961    0.5886    0.5809       241
weighted avg     0.5960    0.5892    0.5811       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.6250    0.7025    0.6615       121
           1     0.6571    0.5750    0.6133       120

    accuracy                         0.6390       241
   macro avg     0.6411    0.6387    0.6374       241
weighted avg     0.6410    0.6390    0.6375  

Epoch: 500
              precision    recall  f1-score   support

           0     0.6504    0.6612    0.6557       121
           1     0.6525    0.6417    0.6471       120

    accuracy                         0.6515       241
   macro avg     0.6515    0.6514    0.6514       241
weighted avg     0.6515    0.6515    0.6514       241

